In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb


creds_path_ar = ["credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [3]:
if WANDB_enable == True:
    wandb.init(project="sota-mafat-base")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]Alexnet_pytorch'

In [4]:
cd {PATH_ROOT}

C:\SOTA\Mafat


In [5]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.data import feat_data, get_data
from src.visualization import specto_vis

from src.models import arch_setup, base_base_model, alex_model

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [6]:
train = get_data.load_data(PATH_DATA+'\\MAFAT RADAR Challenge - Training Set V1')

In [7]:
train_x, train_y, val_x, val_y = get_data.classic_trainval(PATH_DATA)

Please mount drive and set competition_path correctly


In [8]:
humans = train['iq_sweep_burst'][train['target_type']=='human']
animals = train['iq_sweep_burst'][train['target_type']=='animal']

In [9]:
from src.features import specto_feat
samp = animals[0]
specto_feat.calculate_scalogram(samp).shape
#specto_feat.calculate_spectrogram(samp).shape

(32, 125, 7)

In [10]:

samp_dict = pd.DataFrame(train.values()).iloc[:,:15].join(pd.DataFrame(train.keys()), rsuffix = '_').set_index('0_', drop=True).values

samp_dict = {list(train.keys())[i]:samp_dict[i] for i in range(samp_dict.shape[0])}